In [ ]:
from pyspark.sql import SparkSession


In [ ]:
spark = SparkSession.builder.appName("first_paquet").config("spark.jars.packages", "org.postgresql:postgresql:42.7.3").getOrCreate()
df = spark.read.parquet("../data/dataset.parquet")
df.printSchema()
df.show(5)


In [ ]:
#spark.stop()

In [ ]:
df.show(5)


In [ ]:
print(df.count())
len(df.columns)

In [ ]:
type(df)

# Count the number of null values in each columns

In [ ]:
from pyspark.sql.functions import col, sum
df.select([sum(col(c).isNull().cast("int")).alias(c + "_nulles") for c in df.columns]).count()



# check for outliers

In [ ]:
import pyspark as ps
# pdf['fare_amount'].plot.box()


# Deal with null values

In [ ]:
df.na.drop().count()

In [ ]:
df.count()

# Create dure_trajet Column

In [ ]:
type(df)

In [ ]:
import pyspark.sql.functions as sf
df = df.withColumn("dure_trajet", (sf.unix_timestamp(sf.col("tpep_dropoff_datetime"))- sf.unix_timestamp(sf.col("tpep_pickup_datetime")))/60)

In [ ]:
df.show(2)

In [ ]:
df.describe().show(2)

# Search columns  with the same value duplicates over the columns 

In [ ]:
df.select("trip_distance").distinct().count()

# Select the Categorial Columns

In [ ]:
categrial_df= df.select("dure_trajet","VendorID", "RatecodeID", "store_and_fwd_flag","payment_type","PULocationID","DOLocationID")
categrial_df.show(2)

# Analyse the relation between Categorial data and the target

In [ ]:
# categrial_df.plot.countplot(x='VendorID', y='dure_trajet')

from pyspark.sql.functions import count
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql.functions import avg
categorical_cols = ["VendorID", "RatecodeID", "store_and_fwd_flag", "payment_type"]

for col in categorical_cols:
   agg = (
    categrial_df.groupBy(col).agg(avg("dure_trajet").alias("avg_duration"))
)
   pdf = agg.toPandas()    
   plt.figure(figsize= (25,3))
   sns.barplot(data=pdf, x=col, y="avg_duration")
   plt.title(f"Average trip duration by {col}")
   plt.show()


In [ ]:
from pyspark.sql.functions import avg, desc

high_categorial_cols= ["PULocationID","DOLocationID"]
for col in high_categorial_cols:
   agg = (
    categrial_df.groupBy(col).agg(avg("dure_trajet").alias("avg_duration")).orderBy(desc("avg_duration")).limit(20)
    )
   pdf = agg.toPandas()    
   plt.figure(figsize= (25,3))
   sns.barplot(data=pdf, x=col, y="avg_duration")
   plt.title(f"Average trip duration by {col}")
   plt.show()


# Select the Numerical Features

In [ ]:
numerical_df= df.select("dure_trajet","passenger_count", "trip_distance", "fare_amount","extra","mta_tax","tip_amount","tolls_amount","improvement_surcharge","total_amount","congestion_surcharge","Airport_fee","cbd_congestion_fee")
numerical_df.show(2)

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
plt.figure(figsize=(10,7))
vector_col= "assembled_features"
assembler= VectorAssembler(inputCols= numerical_df.columns, outputCol=vector_col, handleInvalid="skip")
print(type(assembler))

df_vector= assembler.transform(numerical_df).select(vector_col)
# df_vector.show(2)
corr_matrix= Correlation.corr(df_vector, vector_col)
matrix_array = corr_matrix.collect()[0][0].toArray()
pd_matrix = pd.DataFrame(matrix_array, columns=numerical_df.columns, index=numerical_df.columns)
# print(matrix_array)
matrix_df= spark.createDataFrame(pd_matrix, numerical_df.columns)

sns.heatmap(pd_matrix, annot=True)




# Filter Data

In [ ]:
# Filter data
from pyspark.sql.functions import col

df_filter= df.filter((col("trip_distance")> 0) & (col("trip_distance")<200 ) & ( col("dure_trajet") > 0) & (col("passenger_count") > 0))
df_filter.show(2)
print(df_filter.count())

# Look for outliers

I decided to delete thoese columns: 

**iqr method**

In [ ]:
from pyspark.sql.functions import col, when, dayofweek, hour, month, unix_timestamp
from pyspark.sql.functions import sin, cos, col, lit
import math
def data_cleaning(df):
      cleaned_df= df.na.drop()
      cleaned_df = cleaned_df.withColumn("dure_trajet", (sf.unix_timestamp(sf.col("tpep_dropoff_datetime"))- sf.unix_timestamp(sf.col("tpep_pickup_datetime")))/60)
      cleaned_df= cleaned_df.filter((col("trip_distance")> 0) & (col("trip_distance")<200 ) & ( col("dure_trajet") > 0) & (col("passenger_count") > 0))
      #drop unicessary columns
      cleaned_df =cleaned_df.drop("fare_amount","mta_tax", "total_amount","cbd_congestion_fee","store_and_fwd_flag", "extra","improvement_surcharge")
      
      
      # cleaned_df = df_filter.withColumn("tpep_dropoff_datetime", (sf.unix_timestamp(sf.col("tpep_dropoff_datetime"))))

      
      #cleaned_df = cleaned_df.withColumn("tpep_pickup_datetime", (sf.unix_timestamp(sf.col("tpep_pickup_datetime"))))
      
      #add date columns

      cleaned_df= cleaned_df.withColumn("dropoff_hour", hour("tpep_dropoff_datetime"))\
                 .withColumn("dropoff_dayofweek", dayofweek("tpep_dropoff_datetime")) \
                 .withColumn("dropoff_month", month("tpep_dropoff_datetime")) \
                 .withColumn("pickup_hour", hour("tpep_pickup_datetime")) \
                 .withColumn("pickup_dayofweek", dayofweek("tpep_pickup_datetime")) \
                 .withColumn("pickup_month", month("tpep_pickup_datetime"))
      cleaned_df=cleaned_df.drop("tpep_dropoff_datetime","tpep_pickup_datetime")
      #am not goign to unclude passenger_count because it have limited values in outliers handling
      numerical_df_cols= cleaned_df.select("dure_trajet","trip_distance","tip_amount","tolls_amount","congestion_surcharge","Airport_fee")

      #handle Outliers
      for feature in numerical_df_cols.columns:
         quartilles = numerical_df_cols.approxQuantile(feature, [0.25,0.50, 0.75],0) #0 err

         iqr = quartilles[2] - quartilles[0]
         uper_bound= quartilles[2] + 1.5 * iqr
         lower_bound= quartilles[0] - 1.5 * iqr

         cleaned_df= cleaned_df.withColumn( 
              feature,
              when(col(feature)>uper_bound , uper_bound) 
              .when(col(feature) < lower_bound , lower_bound)
              .otherwise(col(feature))
            
         )
         
         print(feature, quartilles)
      return cleaned_df
      


In [ ]:
df_clean = data_cleaning(df)


**encode tmestamp features**

In [ ]:
from pyspark.sql.functions import sin, cos, col, lit
import math

def add_cyclical_time_features(df):
    # Precompute constants
    angle_hour = 2 * math.pi / 24
    angle_day = 2 * math.pi / 7
    angle_month = 2 * math.pi / 12

    # Pickup time
    df = df.withColumn("pickup_hour_sin", sin(col("pickup_hour") * lit(angle_hour))) \
           .withColumn("pickup_hour_cos", cos(col("pickup_hour") * lit(angle_hour))) \
           .withColumn("pickup_dayofweek_sin", sin((col("pickup_dayofweek") - lit(1)) * lit(angle_day))) \
           .withColumn("pickup_dayofweek_cos", cos((col("pickup_dayofweek") - lit(1)) * lit(angle_day))) \
           .withColumn("pickup_month_sin", sin((col("pickup_month") - lit(1)) * lit(angle_month))) \
           .withColumn("pickup_month_cos", cos((col("pickup_month") - lit(1)) * lit(angle_month)))

    # Dropoff time
    df = df.withColumn("dropoff_hour_sin", sin(col("dropoff_hour") * lit(angle_hour))) \
           .withColumn("dropoff_hour_cos", cos(col("dropoff_hour") * lit(angle_hour))) \
           .withColumn("dropoff_dayofweek_sin", sin((col("dropoff_dayofweek") - lit(1)) * lit(angle_day))) \
           .withColumn("dropoff_dayofweek_cos", cos((col("dropoff_dayofweek") - lit(1)) * lit(angle_day))) \
           .withColumn("dropoff_month_sin", sin((col("dropoff_month") - lit(1)) * lit(angle_month))) \
           .withColumn("dropoff_month_cos", cos((col("dropoff_month") - lit(1)) * lit(angle_month)))

    # Drop original hour/day/month columns
    df = df.drop("pickup_hour", "pickup_dayofweek", "pickup_month",
                 "dropoff_hour", "dropoff_dayofweek", "dropoff_month")

    return df


In [ ]:
df_clean = add_cyclical_time_features(df_clean)
df_clean.show(2)

In [ ]:
for c in df_clean.columns:
    val = df_clean.filter(col(c).isNull()).count()
    print(f'{c} : {val} nulls')

# download data to postgreSQL

In [ ]:
print(spark.sparkContext._conf.get("spark.jars.packages"))


In [ ]:
df.write \
  .format("jdbc") \
  .option("url", "jdbc:postgresql://localhost:5432/taxi_eta") \
  .option("dbtable", "taxis") \
  .option("user", "postgres") \
  .option("password", "postgres") \
  .option("driver", "org.postgresql.Driver") \
  .mode("overwrite") \
  .save()

In [ ]:
def save_data(df):
    df.write \
  .format("jdbc") \
  .option("url", "jdbc:postgresql://localhost:5432/taxi_eta") \
  .option("dbtable", "taxis") \
  .option("user", "postgres") \
  .option("password", "postgres") \
  .option("driver", "org.postgresql.Driver") \
  .mode("overwrite") \
  .save()


# Normalize

StandardScaler (z-score normalization) is the best choice.

Most numeric features (trip_distance, fare_amount, tip_amount, etc.) have different scales.

Why scaling IDs actually hurts performance

Scaling IDs causes:

Artificial ordering

Artificial distance

In [ ]:
#pyspark Standar scaler
from pyspark.ml.feature import StandardScaler
def normalize(df):
    cols_to_scale=[
        "passenger_count", "trip_distance", "tip_amount", "tolls_amount","congestion_surcharge", "Airport_fee"
    ]
    assembler= VectorAssembler(inputCols= cols_to_scale, outputCol="vectors")
    assembled_df =assembler.transform(df)
    # assembled_df.show(5)
    scaler= StandardScaler(inputCol="vectors", outputCol="scaled_features")
    scaler_df= scaler.fit(assembled_df).transform(assembled_df)
    cols_to_add_to_features= [
          "DOLocationID","PULocationID",
          "RatecodeID", "VendorID",
          "payment_type",
          "scaled_features",
          "pickup_hour_sin",
         "pickup_hour_cos",
        "pickup_dayofweek_sin",
        "pickup_dayofweek_cos",
        "pickup_month_sin",
        "pickup_month_cos",
        "dropoff_hour_sin",
        "dropoff_hour_cos",
        "dropoff_dayofweek_sin",
        "dropoff_dayofweek_cos",
        "dropoff_month_sin",
         "dropoff_month_cos"
         ]
    assembler_vector= VectorAssembler(inputCols=cols_to_add_to_features, outputCol="features")
    assembled_df_vecotr =assembler_vector.transform(scaler_df)
    assembled_df_vecotr =assembled_df_vecotr.select("features", "dure_trajet")
    return assembled_df_vecotr

normalized_df=normalize(df_clean)
normalized_df.show(2)
    

# Split Data

In [ ]:

def split_data(normalized_df):
  train, test= normalized_df.randomSplit([0.7, 0.3])
  return train, test
train, test = split_data(normalized_df)

# Training

In [ ]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
def training_rf(train, test):
  #set spark logs
  # spark.sparkContext.setLogLevel("INFO")
  rf= RandomForestRegressor(featuresCol= "features", labelCol="dure_trajet", predictionCol="prediction_dure",  numTrees=10,  maxDepth=6)
  model=rf.fit(train)
  predictions= model.transform(test)
  evaluator= RegressionEvaluator(labelCol="dure_trajet", predictionCol="prediction_dure", metricName="rmse")
  print(f"remse:  {evaluator.evaluate(predictions)}" )

  evaluator= RegressionEvaluator(labelCol="dure_trajet", predictionCol="prediction_dure", metricName="r2")
  print(f"re : {evaluator.evaluate(predictions)}")

  return model
training_rf(train, test)

## Medallion architecture

## 1. bronze

In [ ]:
#load data fct
def load_data():
    spark = SparkSession.builder.appName("first_paquet").getOrCreate()
    df = spark.read.parquet("../data/dataset.parquet")
load_data()

## 2.Silver

In [ ]:
from pyspark.sql.functions import when 

def data_cleaning(df):
      df_filter= df.filter((col("trip_distance")> 0) & (col("trip_distance")<200 ) & ( col("dure_trajet") > 0) & (col("passenger_count") > 0))
      df_filter =df_filter.drop("fare_amount","mta_tax", "total_amount","cbd_congestion_fee","store_and_fwd_flag", "extra","improvement_surcharge")
      #am not goign to unclude passenger_count because it have limited values
      numerical_df_cols_no_target= df_filter.select("trip_distance","tip_amount","tolls_amount","congestion_surcharge","Airport_fee")
      for feature in numerical_df_cols_no_target.columns:
         quartilles = numerical_df_cols_no_target.approxQuantile(feature, [0.25,0.50, 0.75],0) #0 err
         iqr = quartilles[2] - quartilles[0]
         uper_bound= quartilles[2] + 1.5 * iqr
         lower_bound= quartilles[0] - 1.5 * iqr

         cleaned_df= df.withColumn( 
              feature,
              when(col(feature)>uper_bound , uper_bound) 
              .when(col(feature) < lower_bound , lower_bound)
              .otherwise(col(feature))
         )
         print(feature, quartilles)
      return cleaned_df
      
df_clean = data_cleaning(df)
#df_clean.show(5)